In [77]:
import os, csv, string, re
import pandas as pd

from sklearn import preprocessing, model_selection
from keras.preprocessing import text, sequence
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential
from keras import layers
from tensorflow.keras.optimizers import SGD

import nltk
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [78]:
!tar -xvf /content/webkb-data.gtar.gz

Streaming output truncated to the last 5000 lines.
webkb/other/texas/http:^^www.cs.utexas.edu^users^zhouxiao^institute.html
webkb/other/texas/http:^^www.cs.utexas.edu^users^dastuart^tempjdir^jokes^jokes.html
webkb/other/texas/http:^^www.cs.utexas.edu^users^luxue^company.html
webkb/other/texas/http:^^www.utexas.edu^teamweb
webkb/other/texas/http:^^www.cs.utexas.edu^cgi^rwho^sunghee
webkb/other/texas/http:^^www.cs.utexas.edu^users^emery^fp+oop=haskell^fp+oop=haskell.html
webkb/other/texas/http:^^www.ticam.utexas.edu^Groups^Composites^index.html
webkb/other/texas/http:^^www.cs.utexas.edu^users^bayardo^papers.html
webkb/other/texas/http:^^www.cs.utexas.edu^users^vbb^eosdis-bm.html
webkb/other/texas/http:^^www.cs.utexas.edu^users^jadair^compound.html
webkb/other/texas/http:^^www.cs.utexas.edu^docs^prof.html
webkb/other/texas/http:^^www.cs.utexas.edu^users^boyer^how-to-gnu-at-ut.html
webkb/other/texas/http:^^www.cs.utexas.edu^users^boyer^boyer-photo.html
webkb/other/texas/http:^^www.cs.utexa

In [79]:

# folder = '/content/webkb/'

# print(os.listdir(os.getcwd()))

# for filename in os.listdir(folder):
#   path = '/content/webkb/' + filename + '/'
#   os.chdir(path)
#   print(filename)
#   label = filename
  
#   for filename in os.listdir(path):
#     print("    "+filename)


     

In [80]:

labels = []
directory = '/content/webkb/'
entries = []

for filename in os.listdir(directory):
  # labels.append(filename)
  label = filename
  for filename in os.listdir(directory + label + '/'):
    uni = filename
    for filename in os.listdir(directory + label + '/' + uni + '/'):
      thisdict = {
          "label" : label,
          "uni" : uni,
          "url" : filename
      }
      entries.append(thisdict)
      
  
print(os.listdir(os.getcwd()))

os.chdir('/content')

csv_file = 'web-data.csv'

with open(csv_file, 'w', newline='') as csvfile:
    categories = ['label', 'university', 'url']
    writer = csv.DictWriter(csvfile, fieldnames=categories)
    writer.writeheader()
    for entry in entries:
      writer.writerow({categories[0] : entry["label"], categories[1] : entry["uni"], categories[2] : entry["url"]})
  


['.config', 'webkb-data.gtar.gz', 'webkb', 'web-data.csv', 'sample_data']


In [81]:
data = pd.read_csv(csv_file, encoding="latin-1", on_bad_lines='skip')
print(data[0:10])

   label university                                                url
0  other       misc           http:^^vip.cs.utsa.edu^vip^overview.html
1  other       misc  http:^^vita.mines.colorado.edu:3857^1s^galmasi...
2  other       misc  http:^^darkwing.uoregon.edu^~rhaller^welcome.html
3  other       misc  http:^^cs.nyu.edu^cs^dept_info^course_home_pag...
4  other       misc        http:^^www.cs.wm.edu^~asminer^darkside.html
5  other       misc  http:^^www-mitpress.mit.edu^jrnls-catalog^evol...
6  other       misc                       http:^^www.cica.indiana.edu^
7  other       misc                        ftp:^^ftp.cit.gu.edu.au^pub
8  other       misc        http:^^www.cs.umass.edu^~ramjee^quotes.html
9  other       misc  http:^^paul.rutgers.edu^~deatz^photos^scrapbk....


In [82]:
# Convert all to lowercase, strip and remove punctuations

def preprocess(text):
    text = text.lower() 
    text=text.strip()  
    text=re.compile('<.*?>').sub('', text) 
    text = re.compile('[%s]' % re.escape(string.punctuation)).sub(' ', text)  
    text = re.sub('\s+', ' ', text)  
    # text = re.sub(r'\[[0-9]*\]',' ',text) 
    text=re.sub(r'[^\w\s]', '', str(text).lower().strip())
    # text = re.sub(r'\d',' ',text) 
    text = re.sub(r'\s+',' ',text)
    return text

def stopwords(text):
  words = ["http", "www", "html"]
  # words = ["http", "www", "html", "cs"]
  # words = ["http", "www", "html", "cs", "edu", "vip", "utsa", "overview"]
  a= [i for i in text.split() if i not in words]
  return ' '.join(a)

def tokenization(text):
  return text.split()

wl = WordNetLemmatizer()
 
# This is a helper function to map NTLK position tags
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN
# Tokenize the sentence
def lemmatizer(string):
    word_pos_tags = nltk.pos_tag(word_tokenize(string)) # Get position tags
    a=[wl.lemmatize(tag[0], get_wordnet_pos(tag[1])) for idx, tag in enumerate(word_pos_tags)] # Map the position tag and lemmatize the word/token
    return " ".join(a)

def final_preprocessing(text):
  # return lemmatizer(stopwords(preprocess(text)))
  return tokenization(stopwords(preprocess(text)))

In [83]:
data['clean_url'] = data['url'].apply(lambda x: final_preprocessing(x))
print(data['clean_url'].head)

<bound method NDFrame.head of 0                     [vip, cs, utsa, edu, vip, overview]
1       [vita, mines, colorado, edu, 3857, 1s, galmasi...
2              [darkwing, uoregon, edu, rhaller, welcome]
3       [cs, nyu, edu, cs, dept, info, course, home, p...
4                        [cs, wm, edu, asminer, darkside]
                              ...                        
8277                       [cs, utexas, edu, users, rvdg]
8278    [cs, utexas, edu, users, utcs, report, 1994, p...
8279                        [cs, utexas, edu, users, vin]
8280    [cs, utexas, edu, users, utcs, report, 1995, p...
8281                   [cs, utexas, edu, users, lavender]
Name: clean_url, Length: 8282, dtype: object>


In [84]:
# Label encoding
label_encoder = preprocessing.LabelEncoder()
data['label'] = label_encoder.fit_transform(data['label'])
data['label']

num_class = len(pd.unique(data['label']))
num_class

# Split into training and test
train_x, test_x, train_y, test_y = model_selection.train_test_split(data['clean_url'],
    data['label'], test_size=0.2, shuffle=True)

# print(train_x[0:10])
# print(train_y[0:10])

In [85]:
# Baseline system on new dataset

# Determine the max length of the urls

i = 0
max_length = 0
for i in range(len(data)):
  if(len(data['clean_url'][i]) > max_length):
    max_length = len(data['clean_url'][i])

max_length

17

In [86]:
# word to integer ID tokenizing + padding
tokenizer = text.Tokenizer()
tokenizer.fit_on_texts(data['clean_url'])
word_index = tokenizer.word_index

train_x_seq = sequence.pad_sequences(tokenizer.texts_to_sequences(train_x), maxlen=max_length)
test_x_seq = sequence.pad_sequences(tokenizer.texts_to_sequences(test_x), maxlen=max_length)

# print(train_x_seq)
print(train_x_seq[0])

train_y = to_categorical(train_y, num_class)
test_y = to_categorical(test_y, num_class)

# print(train_x_seq.shape)
# print(train_y)
# print(train_y.shape)

[   0    0    0    0    0    0    0    0    0    2    8    1    6  180
 2381  114 4359]


In [87]:
# Create CNN
vocab_size = len(word_index)+1

model = Sequential()
model.add(layers.Embedding(vocab_size, 100))
model.add(layers.Conv1D(filters=128, kernel_size=5, activation='relu')) 
# Should experiment with kernel size and no of filters


model.add(layers.GlobalAveragePooling1D())
model.add(layers.Dense(32, input_shape=(vocab_size, )))
model.add(layers.Activation(activation='relu'))
model.add(layers.Dropout(0.25))
model.add(layers.Dense(num_class))
model.add(layers.Activation('softmax'))

model.summary()

opt = SGD(learning_rate=0.0001)
model.compile(optimizer=opt, 
              loss='categorical_crossentropy',
              metrics=['accuracy'])

print(train_x_seq.shape)
print(train_y.shape)

history = model.fit(
    train_x_seq,
    train_y,
    epochs=40,
    batch_size=100,
    validation_split=0.1,
    verbose=1)

results = model.evaluate(test_x_seq, test_y)



print(results)

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_6 (Embedding)     (None, None, 100)         715200    
                                                                 
 conv1d_6 (Conv1D)           (None, None, 128)         64128     
                                                                 
 global_average_pooling1d_6   (None, 128)              0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense_12 (Dense)            (None, 32)                4128      
                                                                 
 activation_12 (Activation)  (None, 32)                0         
                                                                 
 dropout_6 (Dropout)         (None, 32)                0         
                                                      

In [88]:
# New Model

# Preprocessing ideas:
# Remove instances without real word inputs in url
# Remove any non-real words from instances
# Remove digits